In [1]:
import torch
import torchvision
import cv2
import matplotlib.pyplot as plt
import numpy as np
import torchvision.transforms as trans
from tqdm import tqdm
from PIL import Image, ImageDraw
import math
import numpy

device = torch.device("cpu")

In [2]:
# accumulate pixel 
class Accumulator:
    def __init__(self, n):
        self.data = [0.0] * n

    def add(self, args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]

    def reset(self):
        self.data = [0.0] * len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [3]:
model = torch.load('best.pt')['model'].to('cpu')
model.eval()

DetectionModel(
  (model): Sequential(
    (0): C_ADD(
      (conv): Conv2d(3, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (1): Conv(
      (conv): Conv2d(4, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): Conv(
      (conv): Conv2d(48, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (3): C2f(
      (cv1): Conv(
        (conv): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(192, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(96, eps=0.001,

In [4]:
# from yolov8
def xywh2xyxy(x):
        assert x.shape[-1] == 4, f'input shape last dimension expected 4 but input shape is {x.shape}'
        y = torch.empty_like(x) if isinstance(x, torch.Tensor) else np.empty_like(x)  # faster than clone/copy
        dw = x[..., 2] / 2  # half-width
        dh = x[..., 3] / 2  # half-height
        y[..., 0] = x[..., 0] - dw  # top left x
        y[..., 1] = x[..., 1] - dh  # top left y
        y[..., 2] = x[..., 0] + dw  # bottom right x
        y[..., 3] = x[..., 1] + dh  # bottom right y
        return y
def non_max_suppression(
            prediction,
            conf_thres=0.001,
            iou_thres=0.8,
            classes=None,
            agnostic=False,
            multi_label=True,
            labels=(),
            max_det=300,
            nc=17,  # number of classes (optional)
            max_time_img=0.05,
            max_nms=30000,
            max_wh=math.inf,
):
        assert 0 <= conf_thres <= 1, f'Invalid Confidence threshold {conf_thres}, valid values are between 0.0 and 1.0'
        assert 0 <= iou_thres <= 1, f'Invalid IoU {iou_thres}, valid values are between 0.0 and 1.0'
        if isinstance(prediction, (list, tuple)):  # YOLOv8 model in validation model, output = (inference_out, loss_out)
            prediction = prediction[0]  # select only inference output
    
        device = prediction.device
        mps = 'mps' in device.type  # Apple MPS
        if mps:  # MPS not fully supported yet, convert tensors to CPU before NMS
            prediction = prediction.cpu()
        bs = prediction.shape[0]  # batch size
        nc = nc or (prediction.shape[1] - 4)  # number of classes
        nm = prediction.shape[1] - nc - 4
        mi = 4 + nc  # mask start index
        xc = prediction[:, 4:mi].amax(1) > conf_thres  # candidates
    
        # Settings
        # min_wh = 2  # (pixels) minimum box width and height
        time_limit = 0.5 + max_time_img * bs  # seconds to quit after
        multi_label &= nc > 1  # multiple labels per box (adds 0.5ms/img)
    
        prediction = prediction.transpose(-1, -2)  # shape(1,84,6300) to shape(1,6300,84)
        prediction[..., :4] = xywh2xyxy(prediction[..., :4])  # xywh to xyxy
        output = [torch.zeros((0, 6 + nm), device=prediction.device)] * bs
        for xi, x in enumerate(prediction):  # image index, image inference
            # Apply constraints
            # x[((x[:, 2:4] < min_wh) | (x[:, 2:4] > max_wh)).any(1), 4] = 0  # width-height
            x = x[xc[xi]]  # confidence
    
            # Cat apriori labels if autolabelling
            if labels and len(labels[xi]):
                lb = labels[xi]
                v = torch.zeros((len(lb), nc + nm + 4), device=x.device)
                v[:, :4] = xywh2xyxy(lb[:, 1:5])  # box
                v[range(len(lb)), lb[:, 0].long() + 4] = 1.0  # cls
                x = torch.cat((x, v), 0)
    
            # If none remain process next image
            if not x.shape[0]:
                continue
    
            # Detections matrix nx6 (xyxy, conf, cls)
            box, cls, mask = x.split((4, nc, nm), 1)
    
            if multi_label:
                i, j = torch.where(cls > conf_thres)
                x = torch.cat((box[i], x[i, 4 + j, None], j[:, None].float(), mask[i]), 1)
            else:  # best class only
                conf, j = cls.max(1, keepdim=True)
                x = torch.cat((box, conf, j.float(), mask), 1)[conf.view(-1) > conf_thres]
    
            # Filter by class
            if classes is not None:
                x = x[(x[:, 5:6] == torch.tensor(classes, device=x.device)).any(1)]
    
            # Check shape
            n = x.shape[0]  # number of boxes
            if not n:  # no boxes
                continue
            if n > max_nms:  # excess boxes
                x = x[x[:, 4].argsort(descending=True)[:max_nms]]  # sort by confidence and remove excess boxes
    
            # Batched NMS
            c = x[:, 5:6] * (0 if agnostic else max_wh)  # classes
            boxes, scores = x[:, :4] + c, x[:, 4]  # boxes (offset by class), scores
            i = torchvision.ops.nms(boxes, scores, iou_thres)  # NMS
            i = i[:max_det]  # limit detections
    
    
            output[xi] = x[i]
            if mps:
                output[xi] = output[xi].to(device)
    
        return output

In [5]:
cityscapesid2trainid = {
    0: -1, 1: -1, 2: -1, 3: -1, 4:-1, 5: -1, 6: -1, 7: 0, 8: 1, 9: 2, 10: 3, 11: 4, 12: 5, 13: 6, 14: 7,15: -1, 16: -1,17: 8, 
    18: -1, 19: 9, 20: 10,21: -1, 22: -1, 23: -1, 24: 11,25: 11, 26: 12,27: 21, 28: 12, 29: 12, 30: 13, 31: 14, 32: 15, 33: 16, -1: -1
}

In [6]:
def target_preprocess(target, mapping):
    for origin in mapping.keys():
        target = torch.where(target == origin, mapping[origin], target)
    return target

In [7]:
def evaluate(class_id, pred, target, padding=(0, (2048-1024)/2)):
    # pred is in the form of(x1, y1, x2, y2, conf, class_id)
    prediction = torch.full(fill_value=-1, size=(target.shape[-1], target.shape[-2]), device=target.device)
    prediction.fill_(-1)
    target = torch.where(target == class_id, class_id, -1)
    pred_ = pred.clone()
    pred_[:, (0, 2)] -= padding[0]
    pred_[:, (1, 3)] -= padding[1]
    for p in pred_:
        if p[-1] == class_id:
            prediction[int(p[1]+0.5):int(p[3]+0.5), int(p[0]+0.5): int(p[2]+0.5)] = class_id
    prediction = prediction.permute([1, 0])
    tp = torch.where((target == class_id)*(prediction ==class_id), 1, 0).sum()
    fp = torch.where((prediction == class_id) * (target == -1), 1, 0).sum()
    tn = torch.where((target == -1) * (prediction == -1), 1, 0).sum()
    fn = target.shape[-1] * target.shape[-2] - tp - fp - tn
    return tp, fp, tn, fn

In [8]:
# from yolov8
def letter_box(img=None):
    shape = img.shape[:2]  # current shape [height, width]
    new_shape = [2048, 2048]

    # Scale ratio (new / old)
    r = min(new_shape[0] / shape[0], new_shape[1] / shape[1])
    r = min(r, 1.0)

    # Compute padding
    ratio = r, r  # width, height ratios
    new_unpad = int(round(shape[1] * r)), int(round(shape[0] * r))
    dw, dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1]  # wh padding
    dw /= 2  # divide padding into 2 sides
    dh /= 2

    if shape[::-1] != new_unpad:  # resize
        img = cv2.resize(img, new_unpad, interpolation=cv2.INTER_LINEAR)
    top, bottom = int(round(dh - 0.1)) , int(round(dh + 0.1))
    left, right = int(round(dw - 0.1)) , int(round(dw + 0.1))
    img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT,
                                 value=(114, 114, 114))  # add border
    return img 

In [9]:
from ultralytics.data.dataset import YOLODataset
from ultralytics.data.build import InfiniteDataLoader
from ultralytics.utils import IterableSimpleNamespace
from pathlib import WindowsPath
from ultralytics.data.utils import PIN_MEMORY

In [11]:
cfg = {'img_path': 'D:\\dev\\ultralytics\\yolo_format\\cityscapes\\val',
       'imgsz': 2048,
       'batch_size': 10,
       'augment': False,
       'hyp':
       IterableSimpleNamespace(
           task='detect',
           mode='val',
           model='D:\\anaconda3\\jupyter notebook dir\\best.pt',
           data='D:\\dev\\ultralytics\\cityscapes.yaml',
           epochs=1,
           patience=50,
           batch=10,
           imgsz=2048,
           save=True,
           save_period=-1,
           cache=False,
           device='cpu',
           workers=0,
           project={},
           name={},
           exist_ok=False,
           pretrained=False,
           optimizer='auto',
           verbose=True,
           seed=0,
           deterministic=True,
           single_cls=False,
           rect=False,
           cos_lr=False,
           close_mosaic=10,
           resume=False,
           fraction=1.0,
           profile=False,
           freeze='None',
           overlap_mask=True,
           mask_ratio=4,
           dropout=0.0,
           val=True,
           split='val',
           save_json=False,
           save_hybrid=False,
           conf=0.5,
           iou=0.7,
           max_det=300,
           half=False,
           dnn=False,
           plots=True,
           source={},
           show=False,
           save_txt=False,
           save_conf=False,
           save_crop=False,
           show_labels=True,
           show_conf=True,
           vid_stride=1,
           stream_buffer=False,
           line_width=8, 
           visualize=False,
           augment=False,
           agnostic_nms=False,
           classes={},
           retina_masks=False,
           boxes=True),
       'rect': False,
       'cache': None,
       'single_cls': False,
       'stride': 32,
       'pad': 0.5,
       'prefix': '\x1b[34m\x1b[1mval: \x1b[0m', 'classes': {},
       'fraction': 1.0
    }
data = {'val': 'D:\\dev\\ultralytics\\yolo_format\\cityscapes\\val',
        'train': 'D:\\',
        'names': {0: 'road', 1: 'sidewalk', 2: 'parking', 3: 'rail track', 4: 'building', 5: 'wall', 6: 'fence', 7: 'guard rail', 8: 'pole', 9: 'traffic light', 10: 'traffic sign', 11: 'person', 12: 'car', 13: 'trailer', 14: 'train', 15: 'motorcycle', 16: 'bicycle'},
        'yaml_file': 'D:\\dev\\ultralytics\\cityscapes.yaml',
        'nc': 17,
        'path': WindowsPath('D:/dev/ultralytics')}

In [12]:
dataset = YOLODataset(
    img_path='D:\\dev\\ultralytics\\yolo_format\\cityscapes\\val',
    imgsz=2048,
    batch_size=1,
    augment=False,
    hyp=cfg['hyp'],
    rect=False,
    cache=None,
    single_cls=False,
    stride=32,
    pad=0.5,
    use_segments=False,
    use_keypoints=False,
    classes=(),
    data=data,
    fraction=1.0
)
dataloader = InfiniteDataLoader(dataset=dataset,
                              batch_size=1,
                              shuffle=False,
                              num_workers=1,
                              sampler=None,
                              pin_memory=PIN_MEMORY,
                              collate_fn=getattr(dataset, 'collate_fn', None),
                              worker_init_fn=None,
                              generator=torch.Generator())

Scanning D:\dev\ultralytics\yolo_format\cityscapes\val\labels.cache... 500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<?, ?it/s]
val: WARNING  D:\dev\ultralytics\yolo_format\cityscapes\val\images\119.png: 1 duplicate labels removed
val: WARNING  D:\dev\ultralytics\yolo_format\cityscapes\val\images\163.png: 1 duplicate labels removed
val: WARNING  D:\dev\ultralytics\yolo_format\cityscapes\val\images\176.png: 1 duplicate labels removed
val: WARNING  D:\dev\ultralytics\yolo_format\cityscapes\val\images\256.png: 1 duplicate labels removed
val: WARNING  D:\dev\ultralytics\yolo_format\cityscapes\val\images\280.png: 1 duplicate labels removed
val: WARNING  D:\dev\ultralytics\yolo_format\cityscapes\val\images\329.png: 1 duplicate labels removed
val: WARNING  D:\dev\ultralytics\yolo_format\cityscapes\val\images\352.png: 2 duplicate labels removed
val: WARNING  D:\dev\ultralytics\yolo_format\cityscapes\val\images\355.png: 1 duplicate labels removed
val: WARNING  D:\dev\ultr

In [10]:
def preprocess(x):
    x = cv2.cvtColor(numpy.asarray(x),cv2.COLOR_RGB2BGR) 
    x = letter_box(x)
    x = trans.ToTensor()(x)
    return x.permute([0, 2, 1])

src_trans = trans.Compose([
    preprocess
])
test_data = torchvision.datasets.Cityscapes(root='D:/anaconda3/Lib/site-packages/cityscapesscripts/download/',
                                           split='val',
                                           mode='fine',
                                           target_type='semantic',
                                           target_transform=trans.ToTensor(),
                                           transform=src_trans)
print(test_data[0][0].device)

cpu


In [81]:
batch_size = 1
preds = None
metrics = [Accumulator(4) for i in range(17)]
total = Accumulator(4)
for idx, batch in tqdm(enumerate(dataloader)):
    preds = model(batch['img'].to(device).half()/255.0)
    res = non_max_suppression(preds)
    for i in range(0, 17):
        target = target_preprocess(test_data[idx][1].permute([0, 2, 1])*255, cityscapesid2trainid).clone().to(device)
        metric = evaluate(i, res[batch_size-1], target)
        total.add(metric)
        metrics[i].add(metric)

500it [56:59,  6.84s/it]


In [82]:
print((0.0 + total[0])/(total[0] + total[1] + total[3]))

0.2708139603883966


In [83]:
def IoU(accmulator):
    return (0.0 + accmulator[0]) / (accmulator[0] + accmulator[1] + accmulator[3])

In [84]:
def PA(accmulator):
    return (0.0 + accmulator[0] + accmulator[2]) / (accmulator[0] + accmulator[1] +accmulator[2] + accmulator[3])

In [85]:
for i in range(17):
    print(f'label {i} : IoU:{IoU(metrics[i])}, PA:{PA(metrics[i])}')

label 0 : IoU:0.48552970388835365, PA:0.7279142150878907
label 1 : IoU:0.08947700874469611, PA:0.8056015195846558
label 2 : IoU:0.00768193270175575, PA:0.96415696144104
label 3 : IoU:0.0, PA:0.9946085271835328
label 4 : IoU:0.27010629767401745, PA:0.6530777482986451
label 5 : IoU:0.007199373473069394, PA:0.9738247804641723
label 6 : IoU:0.020684682738711425, PA:0.9808716325759887
label 7 : IoU:0.0, PA:0.9997403831481934
label 8 : IoU:0.020121417051213557, PA:0.9508402137756348
label 9 : IoU:0.004320607635808001, PA:0.9960992450714111
label 10 : IoU:0.007547349887377524, PA:0.9869889640808105
label 11 : IoU:0.029209640843936544, PA:0.9630387487411499
label 12 : IoU:0.1216936621495841, PA:0.8831227807998657
label 13 : IoU:0.0, PA:0.9996036043167115
label 14 : IoU:0.003194779316257256, PA:0.9965126371383667
label 15 : IoU:8.068672562841587e-05, PA:0.9978963041305542
label 16 : IoU:0.01680004317676792, PA:0.9848852472305298


In [86]:
print(f'Total: IoU:{IoU(total)}, PA:{PA(total)}')

Total: IoU:0.2708139603883966, PA:0.9328696184158325


In [37]:
from neural_compressor import quantization
from neural_compressor.config import PostTrainingQuantConfig

recipes = {
    "smooth_quant": True,
    "smooth_quant_args": {
        "alpha": 0.5,
    },  # default value is 0.5
    "fast_bias_correction": False,
    "max_trial": 10000,
    "time_out": 1000
}

dataloader = torch.utils.data.DataLoader(
    dataset=test_data,
    batch_size=1,
    shuffle=False
)

conf = (
    PostTrainingQuantConfig(recipes=recipes)
)  # default approach is "auto", you can set "dynamic":PostTrainingQuantConfig(approach="dynamic")
q_model = quantization.fit(
    model=model,
    conf=conf,
    calib_dataloader=dataloader,
)
q_model.save("./output")

2023-10-24 18:36:01 [INFO] Start auto tuning.
2023-10-24 18:36:01 [INFO] Quantize model without tuning!
2023-10-24 18:36:01 [INFO] Quantize the model with default configuration without evaluating the model.                To perform the tuning process, please either provide an eval_func or provide an                    eval_dataloader an eval_metric.
2023-10-24 18:36:01 [INFO] Adaptor has 5 recipes.
2023-10-24 18:36:01 [INFO] 1 recipes specified by user.
2023-10-24 18:36:01 [INFO] 3 recipes require future tuning.
2023-10-24 18:36:01 [INFO] *** Initialize auto tuning
Exception in thread 2023-10-24 18:36:01 [INFO] {
2023-10-24 18:36:01 [INFO]     'PostTrainingQuantConfig': {
Thread-15:
Traceback (most recent call last):
  File "D:\anaconda3\lib\threading.py", line 980, in _bootstrap_inner
2023-10-24 18:36:01 [INFO]         'AccuracyCriterion': {
2023-10-24 18:36:01 [INFO]             'criterion': 'relative',
2023-10-24 18:36:01 [INFO]             'higher_is_better': True,
    self.run()
